In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

In [2]:
purchase_history = pd.read_csv('data/raw/purchase_history_datafaker.csv')
customer_interactions = pd.read_csv('data/raw/customer_interactions_datafaker.csv')
product_details = pd.read_csv('data/raw/product_details_datafaker.csv')

In [3]:
# Drop any columns that are completely filled with NaN values
purchase_history.dropna(axis=1, how='all', inplace=True)
product_details.dropna(axis=1, how='all', inplace=True)

In [4]:
customer_interactions

,customer_id,page_views,time_spent
0,20,8,147
1,3,25,122
2,9,38,241
3,8,21,159
4,20,17,279
...,...,...,...
95,13,38,142
96,7,22,291
97,15,23,30
98,11,29,133


In [5]:
purchase_history

,customer_id,product_id,purchase_date
0,22,50,2023-07-12
1,17,72,2024-01-10
2,44,10,2023-06-08
3,50,76,2024-01-06
4,32,14,2023-09-24
...,...,...,...
145,31,47,2023-06-23
146,29,90,2023-11-06
147,33,71,2023-10-13
148,27,64,2024-03-13


In [6]:
product_details

,product_id,category,price,ratings
0,1,Beauty,456,3.2
1,2,Clothing,463,2.8
2,3,Beauty,955,2.4
3,4,Home & Kitchen,103,3.7
4,5,Electronics,820,2.6
...,...,...,...,...
100,101,Beauty,965,1.9
101,102,Beauty,69,4.1
102,103,Home & Kitchen,464,1.1
103,104,Clothing,203,1.7


In [7]:
# Merge datasets
df = pd.merge(purchase_history, customer_interactions, on='customer_id', how='left')
df = pd.merge(df, product_details, on='product_id', how='left')

# Convert purchase_date to datetime and extract useful features
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df['purchase_month'] = df['purchase_date'].dt.month

# Label Encoding for categorical data
df['category'] = df['category'].astype('category').cat.codes

# Assume we create a binary target: whether a customer will purchase again
df['will_purchase_again'] = (df['customer_id'].duplicated(keep=False)).astype(int)

In [8]:
df

,customer_id,product_id,purchase_date,page_views,time_spent,category,price,ratings,purchase_month,will_purchase_again
0,22,50,2023-07-12,NaN,NaN,3,750,3.9,7,1
1,17,72,2024-01-10,33.0,285.0,0,900,4.5,1,1
2,17,72,2024-01-10,42.0,225.0,0,900,4.5,1,1
3,17,72,2024-01-10,49.0,113.0,0,900,4.5,1,1
4,17,72,2024-01-10,35.0,118.0,0,900,4.5,1,1
...,...,...,...,...,...,...,...,...,...,...
432,31,47,2023-06-23,NaN,NaN,0,883,3.0,6,1
433,29,90,2023-11-06,NaN,NaN,2,573,1.2,11,1
434,33,71,2023-10-13,NaN,NaN,3,250,2.3,10,0
435,27,64,2024-03-13,NaN,NaN,1,809,3.8,3,1


In [9]:
X = df[['page_views', 'time_spent', 'price', 'ratings', 'purchase_month', 'category']]
y = df['will_purchase_again']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
print(df['will_purchase_again'].value_counts())

will_purchase_again
1    430
0      7
Name: count, dtype: int64


In [11]:
model = LogisticRegression()
model.fit(X_train, y_train)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values